**qAIntum.ai**

# **Quantum Neural Network (QNN) Classifier**

This note is an example of using Photonic Analog (PA) QNN for binary classification. This is an application of the original work ["Continuous Variable Quantum Neural Networks"](https://arxiv.org/abs/1806.06871).

Compared to Classical Neural Networks, PA QNNs have a reduced number of parameters to train and converge faster with fewer epochs. However, this is a quantum algorithm simulated on classical computers hence the training time for quantum circuits tend to be longder than classical models.

The dataset used in this example can be found https://www.kaggle.com/datasets/shebrahimi/financial-distress.

This file is organized in the following order:
0. Install and import necessary packages
1. Load and preprocess data (The data from Kaggle is saved as 'financial.csv'.
2. Data encoding
3. QNN model
   * QNN layer
   * QNN circuit
   * Model building
4. Model training
5. Evaluation

For the open source repository, refer to https://github.com/qaintumai/quantum.

## **0. Intall Packages**

[Pennylane](https://pennylane.ai/) is a Python based quantum machine learning library by Xanadu. An old version is necessary for this example: v0.29.1.

In [13]:
!pip install -qqq torch torchvision
!pip install -qqq pennylane
!pip install -qqq pennylane-SF

## **0. Import Packages**

In [14]:
import numpy as np
import pandas as pd
import torch
import pennylane as qml
import sklearn

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## **1. Load and Preprocess Data**

Data: Financial Distress
Data points: 3,672
Number of features: 83
Label: real values indicating financial health

For the purpose of this experiment, we are using only 100 data points for training and 20 for testing.

data source: https://www.kaggle.com/datasets/shebrahimi/financial-distress

In [15]:
def load_and_preprocess_data(file_path):
    """
    Load 'financial.csv' and preprocess the data.
    Input: file path
    Output: X_train, X_test, y_train, y_test
    """
    df = pd.read_csv(file_path)
    df = df.drop(['Company', 'Time'], axis=1)
    df.iloc[:, 0][df.iloc[:, 0] > 0.55] = 1.0
    df.iloc[:, 0][df.iloc[:, 0] <= 0.55] = 0.0

    y = df.iloc[:, 0]
    X = df.iloc[:, 1:]

    X = X.to_numpy()
    y = y.to_numpy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    y_scaled = scaler.fit_transform(y.reshape(-1, 1))

    X_train = X[:100]
    X_test = X[100:120]
    y_train = y_scaled[:100]
    y_test = y_scaled[100:120]

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train = torch.tensor(X_train_scaled, requires_grad=True).float()
    X_test = torch.tensor(X_test_scaled, requires_grad=False).float()
    y_train = torch.tensor(np.reshape(y_train, y_train.shape[0]), requires_grad=False).float()
    y_test = torch.tensor(np.reshape(y_test, y_test.shape[0]), requires_grad=False).float()

    return X_train, X_test, y_train, y_test

In [16]:
X_train, X_test, y_train, y_test = load_and_preprocess_data('financial.csv')

/var/folders/16/pyq7sqpn54lf7hj_4295jgnm0000gn/T/ipykernel_57017/840034156.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/16/pyq7sqpn54lf7hj_4295jgnm0000gn/T/ipykernel_57017/840034156.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## **2. Data Encoding**

This step converts classical data into a quantum state by using the data entries as parameters of the quantum gates.

The data encoding gates used are Squeezing, Rotation, Beamsplitter, Displacement Gate, and Kerr Gate. Other gates under "CV operators" in the Pennylane package can be explored.

In [17]:
def data_encoding(x):
    """
    This function converts classical data into a quantum state by using the data entries as parameters of the quantum gates.
    Input: classical data (x)
    No output: quantum state prepared to be operated on by the quantum layers
    """
    num_features = len(x)
    num_wires = 8

    # Squeezing gates
    for i in range(0, min(num_features, num_wires * 2), 2):
        qml.Squeezing(x[i], x[i + 1], wires=i // 2)

    # Beamsplitter gates
    for i in range(num_wires - 1):
        idx = num_wires * 2 + i * 2
        if idx + 1 < num_features:
            qml.Beamsplitter(x[idx], x[idx + 1], wires=[i % num_wires, (i + 1) % num_wires])

    # Rotation gates
    for i in range(num_wires):
        idx = num_wires * 2 + (num_wires - 1) * 2 + i
        if idx < num_features:
            qml.Rotation(x[idx], wires=i)

    # Displacement gates
    for i in range(num_wires):
        idx = num_wires * 2 + (num_wires - 1) * 2 + num_wires + i * 2
        if idx + 1 < num_features:
            qml.Displacement(x[idx], x[idx + 1], wires=i)

    # Kerr gates
    for i in range(num_wires):
        idx = num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + i
        if idx < num_features:
            qml.Kerr(x[idx], wires=i)

    # Squeezing gates (second set)
    for i in range(0, min(num_features - (num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + num_wires), num_wires * 2), 2):
        idx = num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + num_wires + i
        if idx + 1 < num_features:
            qml.Squeezing(x[idx], x[idx + 1], wires=i // 2)

    # Rotation gates (second set)
    for i in range(num_wires):
        idx = num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + num_wires + num_wires * 2 + i
        if idx < num_features:
            qml.Rotation(x[idx], wires=i)

## **3. QNN Model**
To build a model, we need to
* define a layer
* build a circuit with the defined layer
* build a model with the defined circuit.




### **3.1 QNN Layer**
This in a faithful implementation of classical neural networks:
* Weight matrix: Interferometer 1 + Squeezing + Interferometer 2
* Bias addition: Displacement gate
* Nonlinear activation function: Kerr gate

In [18]:
def qnn_layer(v):
    """
    This is one layer of QNN
    Input: quantum parameters (v)
    No output: quantum state prepared to be measured
    """
    num_params = len(v)
    num_wires = 8

    # Interferometer 1
    for i in range(num_wires - 1):
        idx = i * 2
        if idx + 1 < num_params:
            theta = v[idx]
            phi = v[idx + 1]
            qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

    for i in range(num_wires):
        idx = (num_wires - 1) * 2 + i
        if idx < num_params:
            qml.Rotation(v[idx], wires=i)

    # Squeezers
    for i in range(num_wires):
        idx = (num_wires - 1) * 2 + num_wires + i
        if idx < num_params:
            qml.Squeezing(v[idx], 0.0, wires=i)

    # Interferometer 2
    for i in range(num_wires - 1):
        idx = (num_wires - 1) * 2 + num_wires + num_wires + i * 2
        if idx + 1 < num_params:
            theta = v[idx]
            phi = v[idx + 1]
            qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

    for i in range(num_wires):
        idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + i
        if idx < num_params:
            qml.Rotation(v[idx], wires=i)

    # Bias addition
    for i in range(num_wires):
        idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + i
        if idx < num_params:
            qml.Displacement(v[idx], 0.0, wires=i)

    # Non-linear activation function
    for i in range(num_wires):
        idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + num_wires + i
        if idx < num_params:
            qml.Kerr(v[idx], wires=i)

**Weight Initializer**

Randomly initialized values are used as initial parameters of the QNN circuit.

In [19]:
def init_weights(layers, num_wires, active_sd=0.0001, passive_sd=0.1):
    """
    This is a weight vector initializer.
    Input: number of layers, number of wires
    Output: concatenated weight vector
    """
    M = (num_wires-1)*2 + num_wires  # Number of interferometer parameters

    int1_weights = np.random.normal(size=[layers, M], scale=passive_sd)
    s_weights = np.random.normal(size=[layers, num_wires], scale=active_sd)
    int2_weights = np.random.normal(size=[layers, M], scale=passive_sd)
    dr_weights = np.random.normal(size=[layers, num_wires], scale=active_sd)
    k_weights = np.random.normal(size=[layers, num_wires], scale=active_sd)

    weights = np.concatenate([int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)

    return weights

### **3.2 QNN Circuit**

For building a PA circuit as opposed to a qubit-based circuit, we have to choose "strawberryfields.fock" as device.

In [20]:
num_wires = 8
num_basis = 2

# select a device
dev = qml.device("strawberryfields.fock", wires=num_wires, cutoff_dim=num_basis)

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """
    This is a quantum circuit composed of a data encoding circuit and a QNN circuit.
    Input: classical data (inputs), quantum parameters (var)
    Output: quantum state, converted to classical data after measurement
    """
    # convert classical inputs into quantum states
    data_encoding(inputs)

    # iterative quantum layers
    for v in var:
        qnn_layer(v)

    # measure the resulting state and return
    return qml.expval(qml.X(0))

### **3.3 Model building**

In [8]:
num_layers = 2

def get_model(num_wires, num_layers):
    """
    This is a model building function.
    Input: number of modes, number of layers
    Output: PyTorch model
    """
    weights = init_weights(num_layers, num_wires)
    shape_tup = weights.shape
    weight_shapes = {'var': shape_tup}
    qlayer = qml.qnn.TorchLayer(quantum_nn, weight_shapes)
    model = torch.nn.Sequential(qlayer)
    return model

In [9]:
model = get_model(num_wires, num_layers)

## **4. Model Training**

In [10]:
def train_model(model, X_train, y_train, batch_size=5, epochs=6):
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    data_loader = torch.utils.data.DataLoader(
        list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True, drop_last=True
    )

    for epoch in range(epochs):
        running_loss = 0
        for xs, ys in data_loader:
            optimizer.zero_grad()
            x = model(xs).float()
            y = ys.float()
            loss = loss_fn(x, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(data_loader)
        print(f"Average loss over epoch {epoch + 1}: {avg_loss:.4f}")

In [12]:
train_model(model, X_train, y_train, batch_size=5, epochs=3)

Average loss over epoch 1: 0.5400
Average loss over epoch 2: 0.5400
Average loss over epoch 3: 0.5400


## **5. Evaluation**

In [13]:
def evaluate_model(model, X_test, y_test):
    y_pred = model(X_test).detach().numpy()
    y_test = y_test.numpy()
    correct = [1 if p == p_true else 0 for p, p_true in zip(y_pred, y_test)]
    accuracy = sum(correct) / len(y_test)
    print(f"Accuracy: {accuracy * 100}%")

In [14]:
evaluate_model(model, X_test, y_test)

Accuracy: 85.0%
